
#### Reto Boost 2024 ####


#### Datos ####

El objetivo de este reto es construir un algoritmo que ayude a mejorar la distribución de los productos desde los centros de distribución a las tiendas. Para ello, dispondremos de datos reales divididos en dos tablas:
- "workshop_ventas_cd2"
- "workshop_ventas_tienda"

La primera contiene todos los datos que describen el estado de los centros de distribución:
- "id_centro_distribucion": id que hace referencia a cada uno de los almacenes
- "sku_artc": id que hace referencia a los distintos artículos de la compañia (lo que se tiene que distribuir)
- "ventas_previas": Ventas del sku_art acumuladas desde que se lanzó al mercado hasta ahora.
- "cod_family": id que hace referencia a la familia del producto (pantalones, sudaderas etc)
- "precio_base": precio de venta del sku_artc
- "altos/mandas/anchura/cuellos_final": Caracterísitcas descriptivas del sku_artc
- "preds_ecommerce": Predicción de venta online (a nivel global) para ese sku_artc
- "preds_fisicas": Predicción de venta en tienda física (a nivel global) para ese sku_artc
- "cumulative_sales_ecommerce/fisicas": agregado de las predicciones de venta online y fisicas
- "stock": stock disponible de ese artículo en el almacén

La segunda contiene todos los datos que describen el estado de las tiendas que reciben mercancía de cada uno de los centros de distribución:
- "id_centro_distribucion": id que hace referencia a cada uno de los almacenes
- "sku_artc": id que hace referencia a los distintos artículos de la compañia (lo que se tiene que distribuir)
- "id_store": id que hace referencia a las tiendas
- "ventas_previas": Ventas del sku_artc en el id_store acumuladas desde que se lanzó al mercado hasta ahora.
- "max_stock": stock máximo de la tienda
- "store_rfid_store": stock del sku_artc en cuestión en la id_store en cuestión
- "cod_family": id que hace referencia a la familia del producto (pantalones, sudaderas etc)
- "altos/mandas/anchura/cuellos_final": Caracterísitcas descriptivas del sku_artc

#### Objetivo ####

El algoritmo propuesto por los diferentes grupos tendrá que proponer una forma de distribuir el stock de los centros de distribución a las tiendas. Este algoritmo tendrá que distribuir este stock a través de una flota de camiones finita (N, input de la función) con una capacidad finita también (X, input de la función). El algoritmo deberá distribuir este stock de manera que intente, de la mejor forma posible, cumplir con las siguientes condiciones:
- Cumplir con la demanda futura (representada por las columnas de preds_ecommerce y preds_fisicas)
- Maximizar la distribución de artículos "muertos". La vida de los artículos en las tiendas suele ser bastante efímera, venden durante un número de semanas no muy largo (quizás 3-4) y luego empieza a decaer porque se va a acabando el stock o, el mercado ya no demanda ese producto. A través de la venta acumulada y la predicción de la venta restante se puede estimar que artículos están ya al final de su vida, este algoritmo deberá maximizar la sustitución de estos artículos.
- Equlibrio en tienda. Tenemos que mantener un cierto equilibrio en la tienda, que no todos los articulos sean de la misma familia ni del mismo precio etc
- Minimizar canibalización. Tenemos que minimizar el efecto de tener articulos parecidos dentro de la misma tienda.


In [42]:
import pandas as pd

#Tablas disponibles para realizar el workshop
ventas_cd2 = pd.read_parquet('data/workshop_ventas_cd2_v2.parquet')
ventas_tienda = pd.read_parquet('data/workshop_ventas_tienda_v2.parquet')

In [43]:
cd2_filter = ventas_cd2[ventas_cd2['id_centro_distribucion'] == 55596]

tienda_filter = ventas_tienda[ventas_tienda['id_centro_distribucion'] == 55596]
tienda_filter = tienda_filter[tienda_filter['id_store'].isin([37, 57, 61, 62, 65])]

In [44]:
tienda_filter.head()

,id_centro_distribucion,id_store,max_stock,store_rfid_stock,cod_family,precio_base,altos_final,mangas_final,anchura_final,cuellos_final,sku_artc,ventas_previas
1,55596,37,9253,0.0,35,19.950001,None,None,None,None,3614,0.0
2,55596,37,9253,0.0,26,22.950001,None,None,None,None,2981,15.0
5,55596,37,9253,0.0,9,69.949997,None,MANGA LARGA,ANCHA,PICO,5141,2.0
6,55596,37,9253,0.0,21,17.950001,None,None,None,None,2884,14.0
8,55596,37,9253,0.0,1,29.950001,ALTO,None,ANCHO,None,1449,22.0


In [45]:
cd2_filter.head()

,id_centro_distribucion,cod_family,precio_base,altos_final,mangas_final,anchura_final,cuellos_final,preds_ecommerce,cumulative_sales_ecommerce,preds_fisicas,cumulative_sales_fisicas,stock,sku_artc,ventas_previas
0,55596,60,29.950001,None,MANGA LARGA,MEDIA,CAPUCHA,"[58.600777, 61.94627, 91.05056, 91.33501, 73.5...",712.411072,"[0.0, 11.470516, 0.0, 7.9781175, 0.0, 0.0, 0.0...",19.448633,2917.0,3357,134.0
1,55596,3,25.950001,MEDIO,None,LINEA A,None,"[180.23564, 167.52876, 216.35858, 186.59671, 1...",1268.785278,"[28.597677, 7.410217, 10.236149, 0.0, 5.188163...",79.189659,11456.0,2698,902.0
3,55596,1,25.950001,ALTO,None,MOM,None,"[171.62843, 132.08951, 141.70213, 133.93216, 0...",579.352295,"[54.108738, 14.979755, 36.667732, 14.521918, 5...",171.901505,6786.0,2939,751.0
5,55596,9,19.950001,None,MANGA CORTA,AJUSTADO,REDONDO / CAJA,"[250.5542, 263.92807, 244.58339, 254.8803, 222...",2922.527344,"[2790.693, 4161.9263, 3954.5286, 3727.85, 2913...",32165.853516,169005.0,5006,454.0
6,55596,1,29.950001,MEDIO,None,ANCHO,None,"[418.5115, 556.2806, 630.28235, 475.53546, 446...",4898.951172,"[65.26912, 25.17483, 17.811106, 153.22289, 242...",1033.055420,13875.0,3547,386.0


In [86]:
cd2_filter['preds_fisicas'][0]

array([ 0.       , 11.470516 ,  0.       ,  7.9781175,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ], dtype=float32)

In [47]:
def check_product(id_product):
    

SyntaxError: incomplete input (1684453768.py, line 2)

In [48]:
tienda_filter[tienda_filter['sku_artc']==3357]

,id_centro_distribucion,id_store,max_stock,store_rfid_stock,cod_family,precio_base,altos_final,mangas_final,anchura_final,cuellos_final,sku_artc,ventas_previas
314848,55596,62,29908,0.0,60,29.950001,None,MANGA LARGA,MEDIA,CAPUCHA,3357,16.0
548562,55596,57,27127,0.0,60,29.950001,None,MANGA LARGA,MEDIA,CAPUCHA,3357,31.0
628295,55596,65,23506,0.0,60,29.950001,None,MANGA LARGA,MEDIA,CAPUCHA,3357,4.0


In [50]:
cd2_filter[cd2_filter['sku_artc']==3357]

,id_centro_distribucion,cod_family,precio_base,altos_final,mangas_final,anchura_final,cuellos_final,preds_ecommerce,cumulative_sales_ecommerce,preds_fisicas,cumulative_sales_fisicas,stock,sku_artc,ventas_previas
0,55596,60,29.950001,None,MANGA LARGA,MEDIA,CAPUCHA,"[58.600777, 61.94627, 91.05056, 91.33501, 73.5...",712.411072,"[0.0, 11.470516, 0.0, 7.9781175, 0.0, 0.0, 0.0...",19.448633,2917.0,3357,134.0


In [83]:
def espacio_tienda(id_store):
    ocupado = tienda_filter.groupby('id_store')['store_rfid_stock'].sum()[id_store]
    maximo = tienda_filter[tienada_filter['id_store'] == id_store]['max_stock'].unique()[0]
    espacioLibre = maximo-ocupado
    return espacioLibre

In [84]:
espacio_tienda(57)

9618.0